In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import cssutils
import pandas as pd

## PyMongo Web Scrap

In [5]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
# Define database and collection
db = client.mars_db
collection = db.items

In [7]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
# Examine the soup html pull, then determine element that contains sought info
# print(soup.prettify())

In [10]:
# results are returned as an iterable list
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='rollover_description_inner').text
#resource: https://stackoverflow.com/questions/275018/how-can-i-remove-a-trailing-newline
news_title = news_title.strip()
news_p = news_p.strip()


## Scrap Image Using Splinter

In [2]:
# Make sure chromedriver is downloaded to run splinter
# Link: https://sites.google.com/a/chromium.org/chromedriver/downloads

# @NOTE: Replace the path with your actual path to the chromedriver
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [82]:
# Visit the following URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [83]:
# Scrape the browser into soup and use soup to find the full resolution image of mars
# Save the image url to a variable called `featured_image_url `
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#resource: https://stackoverflow.com/questions/24981963/extracting-url-from-style-background-url-with-beautifulsoup-and-without-regex
div_style = soup.find(class_="carousel_item")['style']
style = cssutils.parseStyle(div_style)
image_url = style['background-image']

#Cut of the '()' and 'url'
image_url = image_url.replace('url(', '').replace(')', '')

#added https://www.jpl.nasa.gov to url collected string
featured_image_url = print(f"https://www.jpl.nasa.gov{image_url}")

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16763-1920x1200.jpg


## Scrap Web Facts Using Pandas

In [17]:
# resource: https://stackoverflow.com/questions/47209301/pandas-web-scraping-multiple-pages
mars_facts = []

#  SCRAPE
url = 'https://space-facts.com/mars/'

dframe = pd.read_html(url.format(), header=None)[0]\
                         .rename(columns={0:'Description', 1:'Data'})           
mars_facts.append(dframe)

finaldf = pd.concat(mars_facts)

html_mars_facts = finaldf.to_html(header=False, index=False)
html_mars_facts

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

## Scrap Mars Hemispheres Using Splinter

In [3]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_base = "https://astrogeology.usgs.gov"
browser.visit(url)

title_list = []
for i in range(0,4):
        title_dic = {}
        title = browser.find_by_css('h3')[i].text
        title_dic['title'] = title
        title_list.append(title_dic)
title_list 



[{'title': 'Cerberus Hemisphere Enhanced'},
 {'title': 'Schiaparelli Hemisphere Enhanced'},
 {'title': 'Syrtis Major Hemisphere Enhanced'},
 {'title': 'Valles Marineris Hemisphere Enhanced'}]

In [80]:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_base = "https://astrogeology.usgs.gov"
browser.visit(url)

image = browser.find_by_id('itemLink')
image.click()
# # <a href="/search/map/Mars/Viking/cerberus_enhanced"
# try:
#     browser.click_link_by_partial_href(url_base +"/search/map/Mars/Viking/")
# img

SyntaxError: invalid syntax (<ipython-input-80-71234e5dccd5>, line 8)